In [1]:
# Libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
train = pd.read_csv('data/application_train.csv')
# train = train.sample(10000) #for simple calculations.

In [3]:
# You know, who the hell cares about EDA if you're going to use this for predictions anyway? 
# Who cares about 'why?' if it predicts with 90%?
# You could, of course, use why to identify clients easier, make suggestions for them so they
# know how could they get the loan, but that's for another day.
#
# Let's just XGBoost this thing.
# import xgboost as xgb 

In [26]:
## Data pre-processing.

# Inspecting dtypes.
train.dtypes.unique() # Identifying any weird data formats.
categorical = train.select_dtypes('O') # 16 of categorical variables.
categorical.apply(lambda x: print(x.unique()))

['Cash loans' 'Revolving loans']
['M' 'F' 'XNA']
['N' 'Y']
['Y' 'N']
['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A' nan
 'Other_B' 'Group of people']
['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']
['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
['Single / not married' 'Married' 'Civil marriage' 'Widow' 'Separated'
 'Unknown']
['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']
['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Laborers' 'Realty agents' 'Secretaries'
 'IT staff' 'HR staff']
['WEDNESDAY' 'MONDAY' 'THURSDAY' 'SUNDAY' 'SATURDAY' 'FRIDAY' 'TUESDAY']
['Business Entity Type 3' 'School' 'G

NAME_CONTRACT_TYPE            None
CODE_GENDER                   None
FLAG_OWN_CAR                  None
FLAG_OWN_REALTY               None
NAME_TYPE_SUITE               None
NAME_INCOME_TYPE              None
NAME_EDUCATION_TYPE           None
NAME_FAMILY_STATUS            None
NAME_HOUSING_TYPE             None
OCCUPATION_TYPE               None
WEEKDAY_APPR_PROCESS_START    None
ORGANIZATION_TYPE             None
FONDKAPREMONT_MODE            None
HOUSETYPE_MODE                None
WALLSMATERIAL_MODE            None
EMERGENCYSTATE_MODE           None
dtype: object

In [32]:
# Replacing nan's.
lengths = categorical.apply(lambda x: len(x.unique()))
train[lengths.index] = train[lengths.index].fillna('none')

# Removing post-modern marxists (4 of them, might be an error too).
train = train[~(train['CODE_GENDER'] == 'XNA')]

# Removing maternity leaves. (5)
train = train[~(train['NAME_INCOME_TYPE'] == 'Maternity leave')]

# Removing people who don't know their family status (wtf) (4)
train = train[~(train['NAME_FAMILY_STATUS'] == 'Unknown')]

# Is this going to hurt the accuracy? Probably. Is it big enough for me to care? Probably not.

In [33]:
y = train['TARGET']
train = train.drop('TARGET', axis=1)

In [34]:
# Encoding categorical variables.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
for colname in lengths.index:
    train[colname] = LabelEncoder().fit_transform(train[colname])
    if lengths[colname] > 2:  # Adding variables with > 2 categories.
        dummies = pd.get_dummies(train[colname])
        print(colname, lengths[colname], dummies.shape[1])
        train = pd.concat([train, dummies.iloc[:, 1:]], axis=1) # Avoiding dummy variable.
        print(train.shape[1])
        train = train.drop(colname, axis=1)
        

NAME_TYPE_SUITE 8 8
128
NAME_INCOME_TYPE 7 7
133
NAME_EDUCATION_TYPE 5 5
136
NAME_FAMILY_STATUS 5 5
139
NAME_HOUSING_TYPE 6 6
143
OCCUPATION_TYPE 19 19
160
WEEKDAY_APPR_PROCESS_START 7 7
165
ORGANIZATION_TYPE 58 58
221
FONDKAPREMONT_MODE 5 5
224
HOUSETYPE_MODE 4 4
226
WALLSMATERIAL_MODE 8 8
232
EMERGENCYSTATE_MODE 3 3
233


In [7]:
# Splitting into train-test datasets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:, 2:].values, train.iloc[:, 1], test_size=0.2)

KeyboardInterrupt: 

In [8]:
# Fitting XGBoost to the training set.
from xgboost import XGBClassifier
classifier = XGBClassifier() # Currently satisfied with default parameters.
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
# Comparing predictions with the actual data.
y_pred = classifier.predict(X_test)

C:\Users\Jurgis\AppData\Local\conda\conda\envs\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
print(pd.DataFrame({
    'Predictions': y_pred,
    'Actual Data': y_test
}).head(5))
print('==============================\n')

from sklearn.metrics import confusion_matrix
print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=[0,1]))

        Predictions  Actual Data
199997            0            0
208139            0            1
142289            0            0
298554            0            0
43263             0            0

       0   1
0  56529  25
1   4911  38


In [139]:
# Applying the k-Fold Cross Validation.
# The goal of this is to get the AVERAGE performance of K-folds in the data, instead of a single 
# 'test_set' performance.
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)
print('Mean Accuracy (10-fold): %s%%' % np.round(accuracies.mean() * 100, decimals=2))
print('Standart Accuracy Deviation (10-fold): %s%%' % np.round(accuracies.std() * 100, decimals=2))

C:\Users\Jurgis\AppData\Local\conda\conda\envs\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Jurgis\AppData\Local\conda\conda\envs\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Jurgis\AppData\Local\conda\conda\envs\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Jurgis\AppData\Local\conda\conda\envs\data_science\lib\

Mean Accuracy (10-fold): 91.55%
Standart Accuracy Deviation (10-fold): 0.27%


C:\Users\Jurgis\AppData\Local\conda\conda\envs\data_science\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [35]:
# Submission file block 1.
test = pd.read_csv('data/application_test.csv')

# Getting colnames.
categorical = test.select_dtypes('O') # 16 of categorical variables.

# Replacing nan's.
lengths = categorical.apply(lambda x: len(x.unique()))
test[lengths.index] = test[lengths.index].fillna('none')

# Encoding categorical variables.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
for colname in lengths.index:
    test[colname] = LabelEncoder().fit_transform(test[colname])
    if lengths[colname] > 2:  # Adding variables with > 2 categories.
        dummies = pd.get_dummies(test[colname])
        print(colname, lengths[colname], dummies.shape[1])
        test = pd.concat([test, dummies.iloc[:, 1:]], axis=1) # Avoiding dummy variable.
        print(test.shape[1])
        test = test.drop(colname, axis=1)

NAME_TYPE_SUITE 8 8
128
NAME_INCOME_TYPE 7 7
133
NAME_EDUCATION_TYPE 5 5
136
NAME_FAMILY_STATUS 5 5
139
NAME_HOUSING_TYPE 6 6
143
OCCUPATION_TYPE 19 19
160
WEEKDAY_APPR_PROCESS_START 7 7
165
ORGANIZATION_TYPE 58 58
221
FONDKAPREMONT_MODE 5 5
224
HOUSETYPE_MODE 4 4
226
WALLSMATERIAL_MODE 8 8
232
EMERGENCYSTATE_MODE 3 3
233


In [36]:
# Submission file block 2.
# Fitting XGBoost to the training set.
from xgboost import XGBClassifier
classifier = XGBClassifier() # Currently satisfied with default parameters.
classifier.fit(train.iloc[:, 1:].values, y.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [66]:
# Making the predictions.
y_pred = classifier.predict_proba(test.iloc[:, 1:].values)
submission = pd.DataFrame({'SK_ID_CURR': test.iloc[:, 0].values,
                           'TARGET': np.round([y_val[0] for y_val in y_pred], decimals=1)})

In [68]:
submission.head(2)

,SK_ID_CURR,TARGET
0,100001,0.1
1,100005,0.1


In [65]:
submission.to_csv('submission.csv', index=False)